## Creating table

### 1. without schema

In [0]:

# default schema spark decided

columns = ["emp_id","emp_name","dept","city","salary","joining_date"]
data = [
        (101,"Rahul","IT","Pune",75000,"2020-1-10"),
        (102,"Anita","HR","Mumbai",60000,"2019-3-15"),
        (103,"Vikas","IT","Bangalore",80000,"2021-7-1"),
        (104,"Pooja","Finance","Pune",65000,"2018-11-20"),
        (105,"Amit","HR","Mumbai",55000,"2022-2-5"),
        (106,"Sneha","IT","Pune",90000,"2017-6-30")

        ]

df = spark.createDataFrame(data, columns)


In [0]:
df.printSchema()

### 2. with schema

In [0]:
# schema as we give 

from pyspark.sql.types import *
from datetime import date

schema = StructType([
                    StructField("emp_id", IntegerType(), nullable=False),
                    StructField("emp_name", StringType(), nullable=True),
                    StructField("dept", StringType(), nullable=True),
                    StructField("city", StringType(), nullable=True),
                    StructField("salary", DoubleType(), nullable=True),
                    StructField("joining_date", DateType(),nullable=True)
                    ])
data = [
        (101, "Rahul", "IT", "Pune", 75000, date(2020, 1, 10)),
        (102, "Anita", "HR", "Mumbai", 60000, date(2019, 3, 15)),
        (103, "Vikas", "IT", "Bangalore", 80000, date(2021, 7, 1)),
        (104, "Pooja", "Finance", "Pune", 65000, date(2018, 11, 20)),
        (105, "Amit", "HR", "Mumbai", 55000, date(2022, 2, 5)),
        (106, "Sneha", "IT", "Pune", 90000, date(2017, 6, 30)),
        (107, "Raj", "IT", "Pune", 80000, date(2017, 5, 24)),
    ]

df = spark.createDataFrame(data, schema)

In [0]:
df.printSchema()

In [0]:
df.display()

### save as table

In [0]:
# to save as table in catalog
# df.write.format("delta").mode("overwrite").saveAsTable("employee_records")


### 3. craete table using sql 

In [0]:
# %sql
# CREATE TABLE employee (
#     emp_id INT NOT NULL,
#     emp_name STRING,
#     dept STRING,
#     city STRING,
#     salary DOUBLE,
#     joining_date DATE
# )
# USING DELTA;

# INSERT INTO employee VALUES
# (101, 'Rahul', 'IT', 'Pune', 75000, DATE '2020-01-10'),
# (102, 'Anita', 'HR', 'Mumbai', 60000, DATE '2019-03-15'),
# (103, 'Vikas', 'IT', 'Bangalore', 80000, DATE '2021-07-01'),
# (104, 'Pooja', 'Finance', 'Pune', 65000, DATE '2018-11-20'),
# (105, 'Amit', 'HR', 'Mumbai', 55000, DATE '2022-02-05'),
# (106, 'Sneha', 'IT', 'Pune', 90000, DATE '2017-06-30');


## Transformations

- 1. Give the list of employees who are earning more than 70,000 and working in the same company for the last 3 years.

In [0]:
from pyspark.sql.functions import * 

df1= df.filter((col("salary")>70000) & (col("joining_date")<(add_months(current_date(), -36)) ))
df1.display()
# df1.select("emp_name").display()

- 2. Fetch employees who joined before 2021 and belong to the IT department

In [0]:
df2 = df.filter((col("joining_date") < date(2021,1,1) ) & (col("dept")=="IT") )
df2.display()

- 3. Get employees whose salary is between 50,000 and 80,000.

In [0]:
df3 = df.filter((col("salary") > 50000) & (col("salary") < 80000))
df3.display()

- 4. List employees working in Pune or Mumbai with salary greater than 60,000

In [0]:
df4 = df.filter( (col("city")=="Pune") | (col("city")=="Mumbai") & (col("salary") > 60000) )
df4.display()

- 5.Find employees who joined in the last 2 years.

In [0]:
df5 = df.filter(col("joining_date") < (add_months(current_date() , -24)))
df5.display()

- 6. Get the list of employees whose name starts with ‘A’.

In [0]:
df6 = df.filter(col("emp_name").like("A%"))
df6.display()

- 7. Find employees who are not working in the HR department.

In [0]:
df7 = df.filter(col("dept") != "HR")
df7.display()

- 8. List employees who have the same salary.

In [0]:
a = df.alias("a")
b = df.alias("b")

df8 = a.join(b,col("a.salary") == col("b.salary"),"inner")

df8.filter(col("a.emp_id") != col("b.emp_id")).select("a.emp_name").display()


- 9. Get employees who joined in the year 2020.

In [0]:
df9 = df.filter(col("joining_date").like("2020-%"))
df9.display()

- 10. Give count of employees city-wise.

In [0]:
df10_1 = df.groupBy("city").count()
df10_1.display()

In [0]:
df10_2 = df.groupBy("city").agg(count("*").alias("count_of_employees"))
df10_2.display()

- 11. Find top 3 highest paid employees.

In [0]:
df11 = df.orderBy("salary",ascending=False).limit(3)
display(df11)

- 12. Get employees whose experience is more than 5 years.

In [0]:
df12 = df.filter(col("joining_date") < (add_months(current_date() , -60)))
df12.display()

- 13.Find total salary by city

In [0]:
df13= df.groupBy("city").sum("salary")
df13.display()

In [0]:
df13_1= df.groupBy("city").agg(sum("salary").alias("sum_of_salary"))
df13_1.display()

- 14. Add 10% bonus column

In [0]:
df14= df.withColumn("Bonus",col("salary")*0.1)
df14.display()

- 15. Find average salary per department

In [0]:
df15 = df.groupBy("dept").avg("salary")
df15.display()

In [0]:
df15_1 = df.groupBy("dept").agg(avg("salary").alias("avg_salary"))
df15_1.display()